In [1]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
else:
    try:
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin

import mshr
import fenics as fe

--2023-12-28 14:09:37--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4156 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.06K  --.-KB/s    in 0s      

2023-12-28 14:09:37 (44.5 MB/s) - ‘/tmp/fenics-install.sh’ saved [4156/4156]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/2a03579/releases/pybind11-install.s

In [2]:
def get_length(mesh):

  x_coords = mesh.coordinates()[:, 0]
  length = x_coords.max() - x_coords.min()

  return length

In [4]:
def define_boundaries(mesh):

    length = get_length(mesh)

    class Inflow(fe.SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and fe.near(x[0], 0)

    class Walls(fe.SubDomain):
        def inside(self, x, on_boundary):
          return on_boundary and x[0] > 0 and x[0] < length

    class Outflow(fe.SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and fe.near(x[0], length)

    boundaries = fe.MeshFunction('size_t', mesh, mesh.topology().dim() - 1)
    boundaries.set_all(0)

    inflow_domain = Inflow()
    walls_domain = Walls()
    outflow_domain = Outflow()

    inflow_domain.mark(boundaries, 1)   # Mark inflow with '1'
    walls_domain.mark(boundaries, 2)    # Mark walls with '2'
    outflow_domain.mark(boundaries, 3)  # Mark outflow with '3'

    return boundaries